In [163]:
lai_min = 40.492423
lai_max = 40.918213
long_min = -74.259052
long_max = -73.700809



In [164]:
long_list = []
long_cell =  (long_max - long_min)/50
for i in range(0,50):
    long_list.append(long_min+long_cell*i)
print(long_list)


[-74.259052, -74.24788714, -74.23672228, -74.22555742, -74.21439256, -74.2032277, -74.19206283999999, -74.18089798, -74.16973312, -74.15856826, -74.1474034, -74.13623854, -74.12507368, -74.11390882, -74.10274396, -74.0915791, -74.08041424, -74.06924938, -74.05808452, -74.04691966, -74.0357548, -74.02458994, -74.01342508, -74.00226022, -73.99109536, -73.9799305, -73.96876564, -73.95760078, -73.94643592, -73.93527106, -73.9241062, -73.91294134, -73.90177648000001, -73.89061162, -73.87944676000001, -73.8682819, -73.85711704, -73.84595218, -73.83478732, -73.82362246000001, -73.8124576, -73.80129274000001, -73.79012788, -73.77896302, -73.76779816000001, -73.7566333, -73.74546844000001, -73.73430358, -73.72313872000001, -73.71197386]


In [165]:
lai_list = []
lai_cell =  abs((lai_max - lai_min)/40)
for i in range(0,40):
    lai_list.append(lai_min+lai_cell*i)
print(lai_list)



[40.492423, 40.50306775, 40.513712500000004, 40.52435725, 40.535002000000006, 40.54564675, 40.5562915, 40.566936250000005, 40.577581, 40.58822575, 40.598870500000004, 40.60951525, 40.62016, 40.63080475, 40.6414495, 40.652094250000005, 40.662739, 40.67338375, 40.684028500000004, 40.69467325, 40.705318000000005, 40.71596275, 40.7266075, 40.737252250000004, 40.747897, 40.75854175, 40.7691865, 40.77983125, 40.790476, 40.80112075, 40.8117655, 40.822410250000004, 40.833055, 40.84369975, 40.8543445, 40.86498925, 40.875634000000005, 40.88627875, 40.8969235, 40.907568250000004]


In [166]:
def findSector(laitude,longtitude):
    return(max(i for i in lai_list if i < laitude),max(i for i in long_list if i < longtitude))



crash data set (traffic) 2012-2023 has longtitude and leititude
https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95


policy complain 1961-2022
https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Current-Year-To-Date-/5uac-w243

NYPD shooting 2006-2021
https://data.cityofnewyork.us/Public-Safety/NYPD-Shooting-Incident-Data-Historic-/833y-fsy8

nypd arrest 2006-2021
https://data.cityofnewyork.us/Public-Safety/NYPD-Arrests-Data-Historic-/8h9b-rp9u

find 2012-2021

In [168]:
import sys
import csv
import pandas as pd
import os
from datetime import datetime


In [170]:
start_date = datetime.strptime("01/01/2012", '%m/%d/%Y')
end_date = datetime.strptime("12/31/2021", '%m/%d/%Y')

In [171]:
def findMode(series,keyWord):
    if len(series) == 0:
        return None
    elif len(series) == 1:
        return series.index[0]
    
    if series.index[0] == keyWord:
        return series.index[1]
    else:
        return series.index[0]

Motor_Vehicle_Collisions_Crashes data clean

In [169]:
Motor_Vehicle_Collisions_Crashes_df = pd.read_csv('Motor_Vehicle_Collisions_-_Crashes.csv')


C:\Users\fzhan\AppData\Local\Temp\ipykernel_38760\3890979819.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  Motor_Vehicle_Collisions_Crashes_df = pd.read_csv('Motor_Vehicle_Collisions_-_Crashes.csv')


In [124]:
#Motor_Vehicle_Collisions_Crashes_df
Motor_Vehicle_Collisions_Crashes_map = {}
for i in lai_list:
    for j in long_list:
        Motor_Vehicle_Collisions_Crashes_map[(i,j)] = []

In [125]:
#Motor_Vehicle_Collisions_Crashes_df

for idx, row in Motor_Vehicle_Collisions_Crashes_df.iterrows():
    curr_date = datetime.strptime(row["CRASH DATE"], '%m/%d/%Y')
    if curr_date >= start_date and curr_date <= end_date:
        if pd.isnull(row["LATITUDE"]) == False and pd.isnull(row["LONGITUDE"]) == False and lai_min <= row["LATITUDE"] <= lai_max and long_min <= row["LONGITUDE"] <= long_max:
            curr_sector = findSector(row["LATITUDE"],row["LONGITUDE"])
            if curr_sector in Motor_Vehicle_Collisions_Crashes_map:
                Motor_Vehicle_Collisions_Crashes_map[curr_sector].append([row["NUMBER OF PERSONS INJURED"],
                                                                        row["NUMBER OF PERSONS KILLED"],
                                                                        row["NUMBER OF PEDESTRIANS INJURED"],
                                                                        row["NUMBER OF PEDESTRIANS KILLED"],
                                                                        row["NUMBER OF CYCLIST INJURED"],
                                                                        row["NUMBER OF CYCLIST KILLED"],
                                                                        row["NUMBER OF MOTORIST INJURED"],
                                                                        row["NUMBER OF MOTORIST KILLED"],
                                                                        row["CONTRIBUTING FACTOR VEHICLE 1"],
                                                                        row["CONTRIBUTING FACTOR VEHICLE 2"],
                                                                        row["CONTRIBUTING FACTOR VEHICLE 3"],
                                                                        row["CONTRIBUTING FACTOR VEHICLE 4"],
                                                                        row["CONTRIBUTING FACTOR VEHICLE 5"],
                                                                        row["VEHICLE TYPE CODE 1"],
                                                                        row["VEHICLE TYPE CODE 2"],
                                                                        row["VEHICLE TYPE CODE 3"],
                                                                        row["VEHICLE TYPE CODE 4"],
                                                                        row["VEHICLE TYPE CODE 5"]])


In [160]:
averaged_Motor_Vehicle_Collisions_Crashes_data = []
for i in Motor_Vehicle_Collisions_Crashes_map:
    if len(Motor_Vehicle_Collisions_Crashes_map[i]) >= 1:
        curr_sector_df = pd.DataFrame(Motor_Vehicle_Collisions_Crashes_map[i], columns=["NUMBER OF PERSONS INJURED",
                                                                                        "NUMBER OF PERSONS KILLED",
                                                                                        "NUMBER OF PEDESTRIANS INJURED",
                                                                                        "NUMBER OF PEDESTRIANS KILLED",
                                                                                        "NUMBER OF CYCLIST INJURED",
                                                                                        "NUMBER OF CYCLIST KILLED",
                                                                                        "NUMBER OF MOTORIST INJURED",
                                                                                        "NUMBER OF MOTORIST KILLED",
                                                                                        "CONTRIBUTING FACTOR VEHICLE 1",
                                                                                        "CONTRIBUTING FACTOR VEHICLE 2",
                                                                                        "CONTRIBUTING FACTOR VEHICLE 3",
                                                                                        "CONTRIBUTING FACTOR VEHICLE 4",
                                                                                        "CONTRIBUTING FACTOR VEHICLE 5",
                                                                                        "VEHICLE TYPE CODE 1",
                                                                                        "VEHICLE TYPE CODE 2",
                                                                                        "VEHICLE TYPE CODE 3",
                                                                                        "VEHICLE TYPE CODE 4",
                                                                                        "VEHICLE TYPE CODE 5"])

        curr_data = [i[0],
                     i[1],
                     curr_sector_df["NUMBER OF PERSONS INJURED"].mean(),
                     curr_sector_df["NUMBER OF PERSONS KILLED"].mean(),
                     curr_sector_df["NUMBER OF PEDESTRIANS INJURED"].mean(),
                     curr_sector_df["NUMBER OF PEDESTRIANS KILLED"].mean(),
                     curr_sector_df["NUMBER OF CYCLIST INJURED"].mean(),
                     curr_sector_df["NUMBER OF CYCLIST KILLED"].mean(),
                     curr_sector_df["NUMBER OF MOTORIST INJURED"].mean(),
                     curr_sector_df["NUMBER OF MOTORIST KILLED"].mean(),
                     findMode(curr_sector_df["CONTRIBUTING FACTOR VEHICLE 1"].value_counts(
                     ).nlargest(2), "Unspecified"),
                     findMode(curr_sector_df["CONTRIBUTING FACTOR VEHICLE 2"].value_counts(
                     ).nlargest(2), "Unspecified"),
                     findMode(curr_sector_df["CONTRIBUTING FACTOR VEHICLE 3"].value_counts(
                     ).nlargest(2), "Unspecified"),
                     findMode(curr_sector_df["CONTRIBUTING FACTOR VEHICLE 4"].value_counts(
                     ).nlargest(2), "Unspecified"),
                     findMode(curr_sector_df["CONTRIBUTING FACTOR VEHICLE 5"].value_counts(
                     ).nlargest(2), "Unspecified"),
                     findMode(curr_sector_df["VEHICLE TYPE CODE 1"].value_counts().nlargest(
                         2), None),
                     findMode(curr_sector_df["VEHICLE TYPE CODE 2"].value_counts().nlargest(
                         2), None),
                     findMode(curr_sector_df["VEHICLE TYPE CODE 3"].value_counts().nlargest(
                         2), None),
                     findMode(curr_sector_df["VEHICLE TYPE CODE 4"].value_counts().nlargest(
                         2), None),
                     findMode(curr_sector_df["VEHICLE TYPE CODE 5"].value_counts().nlargest(2), None)]
        
        averaged_Motor_Vehicle_Collisions_Crashes_data.append(curr_data)
        


In [231]:
averaged_Motor_Vehicle_Collisions_Crashes_data_df = pd.DataFrame(averaged_Motor_Vehicle_Collisions_Crashes_data, columns=["sector laitude",
                                                                                                                          "sector longtitude",
                                                                                                                          "Collisions_NUMBER OF PERSONS INJURED",
                                                                                                                          "Collisions_NUMBER OF PERSONS KILLED",
                                                                                                                          "Collisions_NUMBER OF PEDESTRIANS INJURED",
                                                                                                                          "Collisions_NUMBER OF PEDESTRIANS KILLED",
                                                                                                                          "Collisions_NUMBER OF CYCLIST INJURED",
                                                                                                                          "Collisions_NUMBER OF CYCLIST KILLED",
                                                                                                                          "Collisions_NUMBER OF MOTORIST INJURED",
                                                                                                                          "Collisions_NUMBER OF MOTORIST KILLED",
                                                                                                                          "Collisions_CONTRIBUTING FACTOR VEHICLE 1",
                                                                                                                          "Collisions_CONTRIBUTING FACTOR VEHICLE 2",
                                                                                                                          "Collisions_CONTRIBUTING FACTOR VEHICLE 3",
                                                                                                                          "Collisions_CONTRIBUTING FACTOR VEHICLE 4",
                                                                                                                          "Collisions_CONTRIBUTING FACTOR VEHICLE 5",
                                                                                                                          "Collisions_VEHICLE TYPE CODE 1",
                                                                                                                          "Collisions_VEHICLE TYPE CODE 2",
                                                                                                                          "Collisions_VEHICLE TYPE CODE 3",
                                                                                                                          "Collisions_VEHICLE TYPE CODE 4",
                                                                                                                          "Collisions_VEHICLE TYPE CODE 5"])


In [232]:
averaged_Motor_Vehicle_Collisions_Crashes_data_df.to_csv('sector_Motor_Vehicle_Collisions_Crashes_data.csv')


NYPD_Arrests_Data__Historic_ data clean


In [173]:
NYPD_Arrests_Data__Historic_df = pd.read_csv('NYPD_Arrests_Data__Historic_.csv')


In [182]:
NYPD_Arrests_Data__Historic_map = {}
for i in lai_list:
    for j in long_list:
        NYPD_Arrests_Data__Historic_map[(i,j)] = []

In [183]:

for idx, row in NYPD_Arrests_Data__Historic_df.iterrows():
    curr_date = datetime.strptime(row["ARREST_DATE"], '%m/%d/%Y')
    if curr_date >= start_date and curr_date <= end_date:
        if pd.isnull(row["Latitude"]) == False and pd.isnull(row["Longitude"]) == False and lai_min <= row["Latitude"] <= lai_max and long_min <= row["Longitude"] <= long_max:
            curr_sector = findSector(row["Latitude"], row["Longitude"])
            if curr_sector in NYPD_Arrests_Data__Historic_map:
                NYPD_Arrests_Data__Historic_map[curr_sector].append([row["PD_CD"],
                                                                     row["PD_DESC"],
                                                                     row["OFNS_DESC"],
                                                                     row["LAW_CODE"],
                                                                     row["LAW_CAT_CD"],
                                                                     row["ARREST_PRECINCT"],
                                                                     row["JURISDICTION_CODE"],
                                                                     row["AGE_GROUP"],
                                                                     row["PERP_SEX"],
                                                                     row["PERP_RACE"]])


In [200]:
averaged_NYPD_Arrests_Data__Historic = []

for i in NYPD_Arrests_Data__Historic_map:
    if len(NYPD_Arrests_Data__Historic_map[i]) >= 1:

        curr_sector_df = pd.DataFrame(NYPD_Arrests_Data__Historic_map[i], columns=["PD_CD",
                                                                                   "PD_DESC",
                                                                                   "OFNS_DESC",
                                                                                   "LAW_CODE",
                                                                                   "LAW_CAT_CD",
                                                                                   "ARREST_PRECINCT",
                                                                                   "JURISDICTION_CODE",
                                                                                   "AGE_GROUP",
                                                                                   "PERP_SEX",
                                                                                   "PERP_RACE"])

        curr_data = [i[0],
                     i[1],
                     findMode(curr_sector_df["PD_CD"].value_counts(
                     ).nlargest(2), "Unspecified"),
                     findMode(curr_sector_df["PD_DESC"].value_counts(
                     ).nlargest(2), "Unspecified"),
                     findMode(curr_sector_df["OFNS_DESC"].value_counts(
                     ).nlargest(2), "Unspecified"),
                     findMode(curr_sector_df["LAW_CODE"].value_counts(
                     ).nlargest(2), "Unspecified"),
                     findMode(curr_sector_df["LAW_CAT_CD"].value_counts(
                     ).nlargest(2), "Unspecified"),
                     findMode(curr_sector_df["ARREST_PRECINCT"].value_counts(
                     ).nlargest(2), "Unspecified"),
                     findMode(curr_sector_df["JURISDICTION_CODE"].value_counts().nlargest(
                         2), None),
                     findMode(curr_sector_df["AGE_GROUP"].value_counts().nlargest(
                         2), None),
                     findMode(curr_sector_df["PERP_SEX"].value_counts().nlargest(
                         2), None),
                     findMode(curr_sector_df["PERP_RACE"].value_counts().nlargest(
                         2), None)]

        averaged_NYPD_Arrests_Data__Historic.append(curr_data)


In [233]:
averaged_NYPD_Arrests_Data__Historic_df = pd.DataFrame(averaged_NYPD_Arrests_Data__Historic, columns=["sector laitude",
                                                                                                      "sector longtitude",
                                                                                                      "Arrests_PD_CD",
                                                                                                      "Arrests_PD_DESC",
                                                                                                      "Arrests_OFNS_DESC",
                                                                                                      "Arrests_LAW_CODE",
                                                                                                      "Arrests_LAW_CAT_CD",
                                                                                                      "Arrests_ARREST_PRECINCT",
                                                                                                      "Arrests_JURISDICTION_CODE",
                                                                                                      "Arrests_AGE_GROUP",
                                                                                                      "Arrests_PERP_SEX",
                                                                                                      "Arrests_PERP_RACE"])


In [234]:
averaged_NYPD_Arrests_Data__Historic_df.to_csv('sector_NYPD_Arrests_Data__Historic.csv')


NYPD_Complaint data clean

In [194]:
NYPD_Complaint_Data_Current__Year_To_Date_df = pd.read_csv('NYPD_Complaint_Data_Current__Year_To_Date_.csv')


In [198]:
NYPD_Complaint_Data_Current__Year_To_Date_map = {}
for i in lai_list:
    for j in long_list:
        NYPD_Complaint_Data_Current__Year_To_Date_map[(i, j)] = []


for idx, row in NYPD_Complaint_Data_Current__Year_To_Date_df.iterrows():
    curr_date = datetime.strptime(row["CMPLNT_FR_DT"], '%m/%d/%Y')

    if curr_date >= start_date and curr_date <= end_date:
        if pd.isnull(row["Latitude"]) == False and pd.isnull(row["Longitude"]) == False and lai_min <= row["Latitude"] <= lai_max and long_min <= row["Longitude"] <= long_max:
            curr_sector = findSector(row["Latitude"], row["Longitude"])
            if curr_sector in NYPD_Complaint_Data_Current__Year_To_Date_map:
                NYPD_Complaint_Data_Current__Year_To_Date_map[curr_sector].append([row["ADDR_PCT_CD"],
                                                                                   row["CRM_ATPT_CPTD_CD"],
                                                                                   row["JURIS_DESC"],
                                                                                   row["KY_CD"],
                                                                                   row["LAW_CAT_CD"],
                                                                                   row["LOC_OF_OCCUR_DESC"],
                                                                                   row["OFNS_DESC"],
                                                                                   row["PD_CD"],
                                                                                   row["PD_DESC"],
                                                                                   row["PREM_TYP_DESC"],
                                                                                   row["SUSP_AGE_GROUP"],
                                                                                   row["SUSP_RACE"],
                                                                                   row["SUSP_SEX"],
                                                                                   row["VIC_AGE_GROUP"],
                                                                                   row["VIC_RACE"],
                                                                                   row["VIC_SEX"]])


In [210]:
averaged_NYPD_Complaint_Data_Current__Year_To_Date = []

for i in NYPD_Complaint_Data_Current__Year_To_Date_map:
    if len(NYPD_Complaint_Data_Current__Year_To_Date_map[i]) >= 10:

        curr_sector_df = pd.DataFrame(NYPD_Complaint_Data_Current__Year_To_Date_map[i], columns=["ADDR_PCT_CD",
                                                                                                 "CRM_ATPT_CPTD_CD",
                                                                                                 "JURIS_DESC",
                                                                                                 "KY_CD",
                                                                                                 "LAW_CAT_CD",
                                                                                                 "LOC_OF_OCCUR_DESC",
                                                                                                 "OFNS_DESC",
                                                                                                 "PD_CD",
                                                                                                 "PD_DESC",
                                                                                                 "PREM_TYP_DESC",
                                                                                                 "SUSP_AGE_GROUP",
                                                                                                 "SUSP_RACE",
                                                                                                 "SUSP_SEX",
                                                                                                 "VIC_AGE_GROUP",
                                                                                                 "VIC_RACE",
                                                                                                 "VIC_SEX"])

        #this data contians unknown and null, removed null 
        curr_data = [i[0],
                     i[1],
                     findMode(curr_sector_df["ADDR_PCT_CD"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["CRM_ATPT_CPTD_CD"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["JURIS_DESC"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["KY_CD"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["LAW_CAT_CD"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["LOC_OF_OCCUR_DESC"].value_counts(
                     ).nlargest(2), "(null)"),
                    findMode(curr_sector_df["OFNS_DESC"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["PD_CD"].value_counts().nlargest(
                         2), "(null)"),
                     findMode(curr_sector_df["PD_DESC"].value_counts().nlargest(
                         2), "(null)"),
                     findMode(curr_sector_df["PREM_TYP_DESC"].value_counts().nlargest(
                         2), "(null)"),
                     findMode(curr_sector_df["SUSP_AGE_GROUP"].value_counts().nlargest(
                         2), "(null)"),
                     findMode(curr_sector_df["SUSP_RACE"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["SUSP_SEX"].value_counts().nlargest(
                         2), "(null)"),
                     findMode(curr_sector_df["VIC_AGE_GROUP"].value_counts().nlargest(
                         2), "(null)"),
                     findMode(curr_sector_df["VIC_RACE"].value_counts().nlargest(
                         2), "(null)"),
                     findMode(curr_sector_df["VIC_SEX"].value_counts().nlargest(
                         2), "(null)")]


        averaged_NYPD_Complaint_Data_Current__Year_To_Date.append(curr_data)


In [235]:
averaged_NYPD_Complaint_Data_Current__Year_To_Date_df = pd.DataFrame(averaged_NYPD_Complaint_Data_Current__Year_To_Date, columns=["sector laitude",
                                                                                                                                  "sector longtitude",
                                                                                                                                  "Complaint_ADDR_PCT_CD",
                                                                                                                                  "Complaint_CRM_ATPT_CPTD_CD",
                                                                                                                                  "Complaint_JURIS_DESC",
                                                                                                                                  "Complaint_KY_CD",
                                                                                                                                  "Complaint_LAW_CAT_CD",
                                                                                                                                  "Complaint_LOC_OF_OCCUR_DESC",
                                                                                                                                  "Complaint_OFNS_DESC",
                                                                                                                                  "Complaint_PD_CD",
                                                                                                                                  "Complaint_PD_DESC",
                                                                                                                                  "Complaint_PREM_TYP_DESC",
                                                                                                                                  "Complaint_SUSP_AGE_GROUP",
                                                                                                                                  "Complaint_SUSP_RACE",
                                                                                                                                  "Complaint_SUSP_SEX",
                                                                                                                                  "Complaint_VIC_AGE_GROUP",
                                                                                                                                  "Complaint_VIC_RACE",
                                                                                                                                  "Complaint_VIC_SEX"])


In [236]:
averaged_NYPD_Complaint_Data_Current__Year_To_Date_df.to_csv('sector_NYPD_Complaint_Data_Current__Year_To_Date.csv')


NYPD shooting data clean

In [214]:
NYPD_Shooting_Incident_Data__Historic_df = pd.read_csv('NYPD_Shooting_Incident_Data__Historic_.csv')


In [281]:
NYPD_Shooting_Incident_Data__Historic_map = {}
for i in lai_list:
    for j in long_list:
        NYPD_Shooting_Incident_Data__Historic_map[(i, j)] = []


for idx, row in NYPD_Shooting_Incident_Data__Historic_df.iterrows():
    curr_date = datetime.strptime(row["OCCUR_DATE"], '%m/%d/%Y')

    if curr_date >= start_date and curr_date <= end_date:
        if pd.isnull(row["Latitude"]) == False and pd.isnull(row["Longitude"]) == False and lai_min <= row["Latitude"] <= lai_max and long_min <= row["Longitude"] <= long_max:
            curr_sector = findSector(row["Latitude"], row["Longitude"])
            if curr_sector in NYPD_Shooting_Incident_Data__Historic_map:
                NYPD_Shooting_Incident_Data__Historic_map[curr_sector].append([row["PRECINCT"],
                                                                                   row["JURISDICTION_CODE"],
                                                                                   row["LOCATION_DESC"],
                                                                                   row["STATISTICAL_MURDER_FLAG"],
                                                                                   row["PERP_AGE_GROUP"],
                                                                                   row["PERP_SEX"],
                                                                                   row["PERP_RACE"],
                                                                                   row["VIC_AGE_GROUP"],
                                                                                   row["VIC_SEX"],
                                                                                   row["VIC_RACE"]])

In [282]:
averaged_NYPD_Shooting_Incident_Data__Historic = []

for i in NYPD_Shooting_Incident_Data__Historic_map:
    if len(NYPD_Shooting_Incident_Data__Historic_map[i]) >= 10:

        curr_sector_df = pd.DataFrame(NYPD_Shooting_Incident_Data__Historic_map[i], columns=["PRECINCT",
                                                                                             "JURISDICTION_CODE",
                                                                                             "LOCATION_DESC",
                                                                                             "STATISTICAL_MURDER_FLAG",
                                                                                             "PERP_AGE_GROUP",
                                                                                             "PERP_SEX",
                                                                                             "PERP_RACE",
                                                                                             "VIC_AGE_GROUP",
                                                                                             "VIC_SEX",
                                                                                             "VIC_RACE"])

        # this data contians unknown and null, removed null
        curr_data = [i[0],
                     i[1],
                     findMode(curr_sector_df["PRECINCT"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["JURISDICTION_CODE"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["LOCATION_DESC"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["STATISTICAL_MURDER_FLAG"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["PERP_AGE_GROUP"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["PERP_SEX"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["PERP_RACE"].value_counts(
                     ).nlargest(2), "(null)"),
                     findMode(curr_sector_df["VIC_AGE_GROUP"].value_counts().nlargest(
                         2), "(null)"),
                     findMode(curr_sector_df["VIC_SEX"].value_counts().nlargest(
                         2), "(null)"),
                     findMode(curr_sector_df["VIC_RACE"].value_counts().nlargest(
                         2), "(null)")]

        averaged_NYPD_Shooting_Incident_Data__Historic.append(curr_data)


In [283]:
averaged_NYPD_Shooting_Incident_Data__Historic_df = pd.DataFrame(averaged_NYPD_Shooting_Incident_Data__Historic, columns=["sector laitude",
                                                                                                                          "sector longtitude",
                                                                                                                          "Shooting_PRECINCT",
                                                                                                                          "Shooting_JURISDICTION_CODE",
                                                                                                                          "Shooting_LOCATION_DESC",
                                                                                                                          "Shooting_STATISTICAL_MURDER_FLAG",
                                                                                                                          "Shooting_PERP_AGE_GROUP",
                                                                                                                          "Shooting_PERP_SEX",
                                                                                                                          "Shooting_PERP_RACE",
                                                                                                                          "Shooting_VIC_AGE_GROUP",
                                                                                                                          "Shooting_VIC_SEX",
                                                                                                                          "Shooting_VIC_RACE"])


In [284]:
averaged_NYPD_Shooting_Incident_Data__Historic_df.to_csv('sector_NYPD_Shooting_Incident_Data__Historic.csv')


join all datasets


In [292]:
sector_shooting_df = pd.read_csv('sector_NYPD_Shooting_Incident_Data__Historic.csv')
sector_complaint_df = pd.read_csv('sector_NYPD_Complaint_Data_Current__Year_To_Date.csv')
sector_arrest_df = pd.read_csv('sector_NYPD_Arrests_Data__Historic.csv')
sector_collision_df = pd.read_csv('sector_Motor_Vehicle_Collisions_Crashes_data.csv')

In [293]:
sector_list = []
for i in lai_list:
    for j in long_list:
        sector_list.append([i,j])

sector_df = pd.DataFrame(sector_list, columns=["sector laitude","sector longtitude"])

In [294]:
combined_df = pd.merge(sector_df,sector_shooting_df, on=['sector laitude','sector longtitude'], how='outer')
combined_df = pd.merge(combined_df,sector_complaint_df, on=['sector laitude','sector longtitude'], how='outer')
combined_df = pd.merge(combined_df,sector_arrest_df, on=['sector laitude','sector longtitude'], how='outer')
combined_df = pd.merge(combined_df,sector_collision_df, on=['sector laitude','sector longtitude'], how='outer')


C:\Users\fzhan\AppData\Local\Temp\ipykernel_38760\3828397620.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_x'} in the result is deprecated and will raise a MergeError in a future version.
  combined_df = pd.merge(combined_df,sector_collision_df, on=['sector laitude','sector longtitude'], how='outer')


In [295]:

combined_df.shape[0]

2115

In [296]:
count = 0
for idx, row in combined_df.iterrows():
    if [row["sector laitude"], row["sector longtitude"]] not in sector_list:
        print(row)
        break
print(count)


sector laitude                                                         40.59887
sector longtitude                                                    -73.767798
Unnamed: 0_x                                                               14.0
Shooting_PRECINCT                                                         101.0
Shooting_JURISDICTION_CODE                                                  0.0
Shooting_LOCATION_DESC                                  MULTI DWELL - APT BUILD
Shooting_STATISTICAL_MURDER_FLAG                                          False
Shooting_PERP_AGE_GROUP                                                     <18
Shooting_PERP_SEX                                                             M
Shooting_PERP_RACE                                                        BLACK
Shooting_VIC_AGE_GROUP                                                    25-44
Shooting_VIC_SEX                                                              M
Shooting_VIC_RACE                       

In [280]:
# print(lai_list)
# 40.59887
findSector(40.59887,-73.767798)
if findSector(40.59887,-73.767798)[0] in lai_list:
    print("yes")


yes


In [297]:
count = 0
for idx, row in sector_shooting_df.iterrows():
    if [row['sector laitude'],row['sector longtitude']] not in sector_list:
        count = count + 1

print(count)
count = 0
for idx, row in sector_complaint_df.iterrows():
    if [row['sector laitude'],row['sector longtitude']] not in sector_list:
        count = count + 1

print(count)
count = 0
for idx, row in sector_arrest_df.iterrows():
    if [row['sector laitude'],row['sector longtitude']] not in sector_list:
        count = count + 1

print(count)
count = 0
for idx, row in sector_collision_df.iterrows():
    if [row['sector laitude'],row['sector longtitude']] not in sector_list:
        count = count + 1

print(count)

36
48
112
115
